Import necessary packages

In [82]:
from langchain_google_vertexai import ChatVertexAI
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate, MessagesPlaceholder, PromptTemplate
from langchain_openai import ChatOpenAI
from pathlib import Path
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers import PydanticToolsParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from typing import Sequence, List
from langgraph.graph import MessageGraph, END
import asyncio
import nest_asyncio
nest_asyncio.apply()
import pprint

#function to call a path to the file and read it. 
def context_gen(file_name):
    Folder = "VertexLang"
    here = Path(locals().get('__file__', Folder)).resolve()
    parameter = (here / file_name).read_text()
    return parameter

#remove code fences from the output
def remove_code_fences(text):
    lines = text.split("\n")
    lines = [line for line in lines if not line.strip().startswith('```')]
    lines[0] = lines[0].replace(' -', '-', 1)
    print()
    return "\n".join(lines)

#invoke and run the model with the given prompt
def test_vertexai(few_shot_prompt):
    generate = ChatPromptTemplate.from_messages([("system", system), few_shot_prompt, ("human", human), ("ai", someoutput)])
    chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.5)
    #select an output parser
    output_parser = StrOutputParser()
    chain = generate | chat | output_parser
    result = chain.invoke({})
    print(remove_code_fences(result))
    print()
    return

#create inputs to the model, telling it what needs to be done. 
#provides system message
system = context_gen("systemmsg.txt")

#provides the few shot examples
output_examples = context_gen("outputex.txt")

#provides the input examples
input_examples = context_gen("inputex.txt")

#provides the database schema
schema = context_gen("dataBaseSchema.txt")

refsystem = context_gen("ref_system copy.txt")

class SubQuery(BaseModel):
    '''Search for the geometric definition of a feature across the vector database'''

    sub_query: str = Field(
        ...,
        description="The text to be used as a sub-query in the prompt.",
    )   

#First input to the model, breaks query down and provides geometry definition

class YamlText(BaseModel):
    text: str = Field(SQLQuery = "Yaml Text printed one line at a time")


In [83]:
examples = [
    {"input": input_examples, "output": output_examples},
]

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai", "{output}"),
]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)


In [84]:
#This model sends the user query to the LLM, which then breaks it down into smaller parts if there are words it does not understand"""

'''change the human variable to test different inputs'''
human = ("select all faces with a cross sectional area of 10 and then select their corresponding edges")

system2 = context_gen("system2.txt")
prompt = ChatPromptTemplate.from_messages([("system", system2), ("human", human),])

chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.5, )
chat_with_tools = chat.bind_tools([SubQuery])
parser = PydanticToolsParser(tools=[SubQuery])
query_analyzer = prompt | chat_with_tools | parser 
output = query_analyzer.invoke({}, {"tags": ["loop 001"]})
result = output[0].sub_query
print(result)

What is the definition of cross-sectional area?


In [85]:
#The next step is to generate a definition of the words it did not understand, based on the broken down query in the previous step"""

#get context file 
system3 = context_gen("reflect1.txt")

#chat prompt template for defining the words 
definer = ChatPromptTemplate.from_messages([("system", system3), ("human", result)])
chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.5, streaming=True)

#using string output parser as the output is a string
output_parser = StrOutputParser()
chain2 = definer | chat 
definition = chain2.invoke({}, {"tags":["loop 002"]})
print(definition.content)


The cross-sectional area, in terms of CAD geometrical features, can be defined as the area of a planar face that is perpendicular to a specified axis within a solid body.


In [86]:
#Base prompt chain, generates the first yaml version with minimal context
base_prompt = ChatPromptTemplate.from_messages([("system", system), MessagesPlaceholder(variable_name="messages")])
base_chain = base_prompt | chat 

#This chain ise used for the reflection prompt, which asks the model to critique and improve on the generated YAML text.
reflection_prompt = ChatPromptTemplate.from_messages([("system", refsystem), few_shot_prompt, ("ai", definition.content), MessagesPlaceholder(variable_name="messages"), ])
reflect = reflection_prompt | chat 


#The next step is to ask the model to carry out a self reflection on the output that it generated in the previous step and provide feedback on the same, after which it generates an improved output taking the same feedback into consideration."""

reflection_prompt = ChatPromptTemplate.from_messages([("system", refsystem), few_shot_prompt, ("ai", definition.content), MessagesPlaceholder(variable_name="messages"), ])

#chain for the reflection prompt.
reflect = reflection_prompt | chat 



In [87]:
async def generation_node(state: Sequence[BaseMessage]):
    return await base_chain.ainvoke({"messages": state}, {"tags": ["loop 003"]})


async def reflection_node(messages: Sequence[BaseMessage]) -> List[BaseMessage]:
    # Other messages we need to adjust
    cls_map = {"ai": HumanMessage, "human": AIMessage}
    # First message is the original user request. We hold it the same for all nodes
    translated = [messages[0]] + [cls_map[msg.type](content=msg.content) for msg in messages[1:]]
    res = await reflect.ainvoke({"messages": translated})
    # We treat the output of this as human feedback for the generator
    return HumanMessage(content=res.content)

builder = MessageGraph()
builder.add_node("generate", generation_node)
builder.add_node("reflect", reflection_node)
builder.set_entry_point("generate")

def should_continue(state: List[BaseMessage]):
    if len(state) > 5:
        return END
    return "reflect"

builder.add_conditional_edges("generate", should_continue)
builder.add_edge("reflect", "generate")
graph = builder.compile()

async for event in graph.astream([HumanMessage(content=human)]):
    for key, value in event.items():
        if isinstance(value, dict):
            print(value.get('content'))
            print('---')
            print()
        elif isinstance(value, list):
             for item in value:
                if isinstance(item, dict):
                    print(item.get('content'))
                    print('---')
                    print()
                else:
                    print(item.content)
                    print('---')
                    print()
                    


select all faces with a cross sectional area of 10 and then select their corresponding edges
---

- filter: select * from faces where area = 10
- expand: faceedges
---

- filter: select * from faces where area = 10
- expand: faceedges
---

- filter: select * from faces where area = 10
- expand: faceedges
---

- filter: select * from faces where area = 10
- expand: faceedges
---

- filter: select * from faces where area = 10
- expand: faceedges
---

